In [17]:
import pandas as pd
import numpy as np

In [18]:
# Load your merged dataset
df = pd.read_csv('merged_social_determinants_analysis.csv')

In [19]:
print("=== STEP 1A: HOSPITAL-LEVEL VARIABLES ANALYSIS ===\n")

# Task 1A.1: Frequency distributions for categorical variables
print("1. CATEGORICAL VARIABLES - FREQUENCY DISTRIBUTIONS")
print("="*50)

print("\nHospital_Type_Category:")
print(df['Hospital_Type_Category'].value_counts())
print(f"Missing values: {df['Hospital_Type_Category'].isna().sum()}")

print("\nOwnership_Category:")
print(df['Ownership_Category'].value_counts())
print(f"Missing values: {df['Ownership_Category'].isna().sum()}")

print("\nHas_Emergency_Services:")
print(df['Has_Emergency_Services'].value_counts())
print(f"Missing values: {df['Has_Emergency_Services'].isna().sum()}")

# Task 1A.1: Descriptive statistics for continuous variables
print("\n\n2. CONTINUOUS VARIABLES - DESCRIPTIVE STATISTICS")
print("="*50)

print("\nHospital_Rating_Numeric:")
print(df['Hospital_Rating_Numeric'].describe())
print(f"Missing values: {df['Hospital_Rating_Numeric'].isna().sum()}")

=== STEP 1A: HOSPITAL-LEVEL VARIABLES ANALYSIS ===

1. CATEGORICAL VARIABLES - FREQUENCY DISTRIBUTIONS

Hospital_Type_Category:
Hospital_Type_Category
Acute Care    2237
Name: count, dtype: int64
Missing values: 0

Ownership_Category:
Ownership_Category
Private       1543
Non-Profit     372
Public         311
Other           11
Name: count, dtype: int64
Missing values: 0

Has_Emergency_Services:
Has_Emergency_Services
1    2151
0      86
Name: count, dtype: int64
Missing values: 0


2. CONTINUOUS VARIABLES - DESCRIPTIVE STATISTICS

Hospital_Rating_Numeric:
count    2027.000000
mean        3.112481
std         1.160036
min         1.000000
25%         2.000000
50%         3.000000
75%         4.000000
max         5.000000
Name: Hospital_Rating_Numeric, dtype: float64
Missing values: 210


In [20]:
# Task 1A.1: Cross-tabulations
print("\n\n3. CROSS-TABULATIONS")
print("="*50)

print("\nOwnership_Category vs Has_Emergency_Services:")
crosstab1 = pd.crosstab(df['Ownership_Category'], df['Has_Emergency_Services'], margins=True)
print(crosstab1)

print("\nHospital_Type_Category vs Ownership_Category:")
crosstab2 = pd.crosstab(df['Hospital_Type_Category'], df['Ownership_Category'], margins=True)
print(crosstab2)



3. CROSS-TABULATIONS

Ownership_Category vs Has_Emergency_Services:
Has_Emergency_Services   0     1   All
Ownership_Category                    
Non-Profit              36   336   372
Other                    0    11    11
Private                 31  1512  1543
Public                  19   292   311
All                     86  2151  2237

Hospital_Type_Category vs Ownership_Category:
Ownership_Category      Non-Profit  Other  Private  Public   All
Hospital_Type_Category                                          
Acute Care                     372     11     1543     311  2237
All                            372     11     1543     311  2237


In [21]:


# Task 1A.2: Primary outcome distribution
print("\n\n4. PRIMARY OUTCOME VARIABLE")
print("="*50)

print("\nExcess Readmission Ratio:")
print(df['Excess Readmission Ratio'].describe())
print(f"Missing values: {df['Excess Readmission Ratio'].isna().sum()}")

# Check for extreme outliers
print(f"\nOutlier check (values beyond 3 standard deviations):")
mean_err = df['Excess Readmission Ratio'].mean()
std_err = df['Excess Readmission Ratio'].std()
outliers = df[(df['Excess Readmission Ratio'] < mean_err - 3*std_err) | 
              (df['Excess Readmission Ratio'] > mean_err + 3*std_err)]
print(f"Number of potential outliers: {len(outliers)}")

# Additional check for the range mentioned in your summary
print(f"\nExpected range check (your summary mentioned 0.780-1.287):")
print(f"Actual min: {df['Excess Readmission Ratio'].min()}")
print(f"Actual max: {df['Excess Readmission Ratio'].max()}")

print("\n\n5. SAMPLE SIZE CONFIRMATION")
print("="*50)
print(f"Total hospitals in dataset: {len(df)}")
print(f"Complete cases for analysis: {len(df.dropna(subset=['Hospital_Type_Category', 'Ownership_Category', 'Has_Emergency_Services', 'Hospital_Rating_Numeric', 'Excess Readmission Ratio']))}")

# Verify the Measure Name filter
print(f"\nMeasure Name verification:")
print(f"Unique values in Measure Name: {df['Measure Name'].unique()}")
print(f"Count of 'READM-30-PN-HRRP' records: {(df['Measure Name'] == 'READM-30-PN-HRRP').sum()}")



4. PRIMARY OUTCOME VARIABLE

Excess Readmission Ratio:
count    2237.000000
mean        0.999276
std         0.063793
min         0.780300
25%         0.958300
50%         0.995500
75%         1.036900
max         1.287100
Name: Excess Readmission Ratio, dtype: float64
Missing values: 0

Outlier check (values beyond 3 standard deviations):
Number of potential outliers: 15

Expected range check (your summary mentioned 0.780-1.287):
Actual min: 0.7803
Actual max: 1.2871


5. SAMPLE SIZE CONFIRMATION
Total hospitals in dataset: 2237
Complete cases for analysis: 2027

Measure Name verification:
Unique values in Measure Name: ['READM-30-PN-HRRP']
Count of 'READM-30-PN-HRRP' records: 2237


In [22]:
import pandas as pd
import numpy as np

# Load your merged dataset
df = pd.read_csv('merged_social_determinants_analysis.csv')

print("=== STEP 1C: DATA CLEANING FOR LEVEL 1 VARIABLES ===\n")

print("BEFORE CLEANING:")
print("="*30)
print(f"Total sample size: {len(df)}")
print(f"Ownership_Category distribution:")
print(df['Ownership_Category'].value_counts())
print(f"Hospital_Rating_Numeric missing: {df['Hospital_Rating_Numeric'].isna().sum()}")

# Task 1C.1: Handle "Other" ownership category
print("\n\n1. HANDLING 'OTHER' OWNERSHIP CATEGORY")
print("="*50)

# Combine "Other" with "Private" (most common category)
df['Ownership_Category_Clean'] = df['Ownership_Category'].replace('Other', 'Private')

print("After combining 'Other' with 'Private':")
print(df['Ownership_Category_Clean'].value_counts())

# Task 1C.2: Handle missing Hospital_Rating_Numeric values
print("\n\n2. HANDLING MISSING HOSPITAL_RATING_NUMERIC VALUES")
print("="*50)

# Option 1: Create indicator variable for missing ratings
df['Rating_Missing'] = df['Hospital_Rating_Numeric'].isna().astype(int)

# Option 2: Mean imputation (we'll also create this version)
mean_rating = df['Hospital_Rating_Numeric'].mean()
df['Hospital_Rating_Numeric_Imputed'] = df['Hospital_Rating_Numeric'].fillna(mean_rating)

print(f"Created missing indicator: {df['Rating_Missing'].sum()} hospitals with missing ratings")
print(f"Mean imputation value: {mean_rating:.2f}")

# Task 1C.3: Create final Level 1 variables
print("\n\n3. FINAL LEVEL 1 VARIABLES SUMMARY")
print("="*50)

# Final variable selection
level1_vars = ['Ownership_Category_Clean', 'Hospital_Rating_Numeric_Imputed', 'Rating_Missing']

print("Selected Level 1 (Hospital) variables:")
for var in level1_vars:
    print(f"- {var}")

# Check final distributions
print(f"\nFinal Ownership_Category_Clean distribution:")
print(df['Ownership_Category_Clean'].value_counts())

print(f"\nFinal Hospital_Rating_Numeric_Imputed distribution:")
print(df['Hospital_Rating_Numeric_Imputed'].describe())

print(f"\nRating_Missing distribution:")
print(df['Rating_Missing'].value_counts())

# Task 1C.4: Check complete cases after cleaning
print("\n\n4. COMPLETE CASES CHECK")
print("="*50)

complete_cases = df[level1_vars + ['Excess Readmission Ratio']].dropna()
print(f"Complete cases for Level 1 analysis: {len(complete_cases)}")
print(f"Percentage of complete cases: {len(complete_cases)/len(df)*100:.1f}%")

# Task 1C.5: Save cleaned dataset (optional)
print("\n\n5. SAVE CLEANED DATASET")
print("="*50)

# Save the cleaned dataset
df.to_csv('merged_social_determinants_analysis_level1_cleaned.csv', index=False)
print("Cleaned dataset saved as: merged_social_determinants_analysis_level1_cleaned.csv")

print("\n\nSTEP 1C COMPLETED SUCCESSFULLY!")
print("="*50)
print("Ready for Step 2: Level 2 (County) Variable Selection")

=== STEP 1C: DATA CLEANING FOR LEVEL 1 VARIABLES ===

BEFORE CLEANING:
Total sample size: 2237
Ownership_Category distribution:
Ownership_Category
Private       1543
Non-Profit     372
Public         311
Other           11
Name: count, dtype: int64
Hospital_Rating_Numeric missing: 210


1. HANDLING 'OTHER' OWNERSHIP CATEGORY
After combining 'Other' with 'Private':
Ownership_Category_Clean
Private       1554
Non-Profit     372
Public         311
Name: count, dtype: int64


2. HANDLING MISSING HOSPITAL_RATING_NUMERIC VALUES
Created missing indicator: 210 hospitals with missing ratings
Mean imputation value: 3.11


3. FINAL LEVEL 1 VARIABLES SUMMARY
Selected Level 1 (Hospital) variables:
- Ownership_Category_Clean
- Hospital_Rating_Numeric_Imputed
- Rating_Missing

Final Ownership_Category_Clean distribution:
Ownership_Category_Clean
Private       1554
Non-Profit     372
Public         311
Name: count, dtype: int64

Final Hospital_Rating_Numeric_Imputed distribution:
count    2237.000000


## Level 2 County Variable Selection

In [23]:
# Load the cleaned dataset
df = pd.read_csv('merged_social_determinants_analysis_level1_cleaned.csv')

print("=== STEP 2D: FINAL LEVEL 2 VARIABLE SELECTION & CLEANING ===\n")

# Define final Level 2 (County) variables
level2_vars = [
    # Economic (2 variables)
    'median_household_income_raw_value',
    'children_in_poverty_raw_value',
    
    # Healthcare Access (2 variables)
    'uninsured_adults_raw_value',
    'ratio_of_population_to_primary_care_physicians',
    
    # Demographics (2 variables)
    '%_rural_raw_value',
    '%_non_hispanic_white_raw_value',
    
    # Education (1 variable)
    'some_college_raw_value'
]

print("FINAL LEVEL 2 (COUNTY) VARIABLES:")
print("="*50)
for i, var in enumerate(level2_vars, 1):
    print(f"{i}. {var}")

# Check missing data pattern
print("\n\nMISSING DATA ANALYSIS:")
print("="*50)
missing_summary = df[level2_vars].isnull().sum()
print(missing_summary)

# Check if missing data is from same counties
print(f"\nHospitals with any missing Level 2 data: {df[level2_vars].isnull().any(axis=1).sum()}")

# Task 2D.1: Handle missing data
print("\n\n1. HANDLING MISSING DATA")
print("="*50)

# Option 1: Complete case analysis (recommended for multilevel models)
df_complete = df.dropna(subset=level2_vars + ['Excess Readmission Ratio'])

print(f"Original sample: {len(df)} hospitals")
print(f"Complete cases: {len(df_complete)} hospitals")
print(f"Percentage retained: {len(df_complete)/len(df)*100:.1f}%")

# Task 2D.2: Create county grouping variable
print("\n\n2. COUNTY GROUPING VARIABLE")
print("="*50)

# Use county_fips_code as the grouping variable
df_complete['county_group'] = df_complete['county_fips_code']

print(f"Number of counties in final sample: {df_complete['county_group'].nunique()}")
print(f"Average hospitals per county: {len(df_complete) / df_complete['county_group'].nunique():.2f}")

# Check county distribution
county_counts = df_complete['county_group'].value_counts()
print(f"Counties with 1 hospital: {(county_counts == 1).sum()}")
print(f"Counties with 2-5 hospitals: {((county_counts >= 2) & (county_counts <= 5)).sum()}")
print(f"Counties with 6+ hospitals: {(county_counts >= 6).sum()}")

# Task 2D.3: Standardize continuous variables
print("\n\n3. STANDARDIZING CONTINUOUS VARIABLES")
print("="*50)

# Standardize Level 2 variables (z-score)
level2_vars_std = []
for var in level2_vars:
    var_std = var + '_std'
    df_complete[var_std] = (df_complete[var] - df_complete[var].mean()) / df_complete[var].std()
    level2_vars_std.append(var_std)
    print(f"Standardized {var} -> {var_std}")

# Also standardize Level 1 continuous variable
df_complete['Hospital_Rating_Numeric_Imputed_std'] = (
    df_complete['Hospital_Rating_Numeric_Imputed'] - df_complete['Hospital_Rating_Numeric_Imputed'].mean()
) / df_complete['Hospital_Rating_Numeric_Imputed'].std()

# Task 2D.4: Final variable summary
print("\n\n4. FINAL HIERARCHICAL MODEL VARIABLES")
print("="*50)

level1_vars_final = [
    'Ownership_Category_Clean',
    'Hospital_Rating_Numeric_Imputed_std',
    'Rating_Missing'
]

print("LEVEL 1 (HOSPITAL) VARIABLES:")
for var in level1_vars_final:
    print(f"  - {var}")

print("\nLEVEL 2 (COUNTY) VARIABLES:")
for var in level2_vars_std:
    print(f"  - {var}")

print(f"\nCOUNTY GROUPING VARIABLE: county_group")
print(f"PRIMARY OUTCOME: Excess Readmission Ratio")

# Task 2D.5: Save final dataset
print("\n\n5. SAVE FINAL DATASET")
print("="*50)

final_vars = level1_vars_final + level2_vars_std + ['county_group', 'Excess Readmission Ratio', 'Facility ID', 'Facility Name', 'State']
df_final = df_complete[final_vars].copy()

df_final.to_csv('hierarchical_model_final_dataset.csv', index=False)
print(f"Final dataset saved: hierarchical_model_final_dataset.csv")
print(f"Final sample size: {len(df_final)} hospitals")
print(f"Final county count: {df_final['county_group'].nunique()} counties")

print("\n\nSTEP 2D COMPLETED - READY FOR HIERARCHICAL MODELING!")

=== STEP 2D: FINAL LEVEL 2 VARIABLE SELECTION & CLEANING ===

FINAL LEVEL 2 (COUNTY) VARIABLES:
1. median_household_income_raw_value
2. children_in_poverty_raw_value
3. uninsured_adults_raw_value
4. ratio_of_population_to_primary_care_physicians
5. %_rural_raw_value
6. %_non_hispanic_white_raw_value
7. some_college_raw_value


MISSING DATA ANALYSIS:
median_household_income_raw_value                 83
children_in_poverty_raw_value                     83
uninsured_adults_raw_value                        83
ratio_of_population_to_primary_care_physicians    84
%_rural_raw_value                                 84
%_non_hispanic_white_raw_value                    83
some_college_raw_value                            83
dtype: int64

Hospitals with any missing Level 2 data: 85


1. HANDLING MISSING DATA
Original sample: 2237 hospitals
Complete cases: 2152 hospitals
Percentage retained: 96.2%


2. COUNTY GROUPING VARIABLE
Number of counties in final sample: 178
Average hospitals per county: 12

/var/tmp/ipykernel_2798/800756071.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_complete['county_group'] = df_complete['county_fips_code']
/var/tmp/ipykernel_2798/800756071.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_complete[var_std] = (df_complete[var] - df_complete[var].mean()) / df_complete[var].std()
/var/tmp/ipykernel_2798/800756071.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value in